# Aggregate vehicle counts by Community Area

by Luc Anselin (anselin@uchicago.edu) (8/23/2016)

We are going to count how many abandoned vehicles are in each Community Area. The
community area ID happens to be part of the vehicle points layer, so that we don't
have to carry out an explicit spatial join (that would be an alternative approach).
The end result is a data frame with the Community Area ID and the count of abandoned
vehicles that we can then join with the Community Area polygon layer in GeoDa.

Note: this is written with R beginners in mind, more seasoned R users can probably skip most of the comments.

For more extensive details about each function, see the R (or RStudio) help files.

Packages used:

- **foreign**

### Create a data frame with the vehicle point observations

We will use the **foreign** package to read the **dbf** file associated with the points data layer 
**abandoned15_9_pts** that was created in GeoDa. Note: it doesn't matter whether we use the file with the
point ID variable or without, since the latter will not be used. However, it is important that you use the file where
the missing Community Area code (area 53 for observation 696) was fixed.

We use **read.dbf** and specify the point layer as the input file. As before, we use **head** to make sure all is OK.

In [1]:
library(foreign)
vpoints <- read.dbf("abandoned15_9_pts_1.dbf")
head(vpoints)

year,month,credate,Ward,Polic_rict,Commu_Area,Latitude,Longitude
2015,9,2015-09-01,3,9,37,41.79584,-87.63289
2015,9,2015-09-01,49,24,1,42.01450,-87.67822
2015,9,2015-09-01,27,12,28,41.88544,-87.66681
2015,9,2015-09-02,4,2,36,41.81816,-87.60088
2015,9,2015-09-02,5,3,43,41.76780,-87.58582
2015,9,2015-09-02,26,12,24,41.89728,-87.68703


### Calculate the point count by Community Area

There are many ways to carry out this calculation in R, including the **aggregate** function and its counterpart in some specialized packages. However, in this instance, since we only count the number of events in a particular category, this is easily accomplished using the **table** command. This yields a contingency table that lists how
many observations are in each category, where the Community Area serves as the category.

After the **table** command, we turn the resulting data object into a data frame with the **as.data.frame** command.

In [2]:
pcounts <- table(vpoints$Commu_Area)
pcounts


  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20 
 42  50  17  29  49  40  28   9  12  28  40   8  14  58 104  64  84  29  69  17 
 21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40 
 29  71  47  76  98  12  13  31  15  32  20  30   1   7  14   7   3  17  10  11 
 41  42  43  44  45  46  48  49  50  51  52  53  55  56  57  58  59  60  61  62 
  6   6  34  35  11  21  15  29   7  12  16  16   5  30  13  48  23  30  45  31 
 63  64  65  66  67  68  69  70  71  72  73  74  75  76  77 
 39  22  33  78  27  22  39  43  32   6  37   3  21   7  41 

In [3]:
pcframe <- as.data.frame(pcounts)
head(pcframe)

Var1,Freq
1,42
2,50
3,17
4,29
5,49
6,40


### Gotcha

This all looks fine, except that there is a problem. Upon closer examination of the table, you can see that
Areas 47 (Burnside) and 54 (Riverdale) don't appear in the table. We can also conclude this from 
applying the **dim** command to the data frame: it has only 75 rows, whereas there are 77 neighborhoods.
Apparently, no vehicles were abandoned in those 
Areas (we leave aside the question whether this is real or a coding error).

In [4]:
dim(pcframe)

[1] 75  2

In and of itself, the mismatch is not a problem, but the point is to illustrate how to deal with this issue.

Basically, we need to end up with a data frame that has all the IDs for the Community Areas and a value of zero
for the counts in Area 47 and 54.

One approach is to initialize a 77 by 1 vector with zero and then extract the values of the vehicle counts from
the data frame, using the Community Area ID as row numbers. Af first sight, this would be a straightforward application of the subsetting command **[ ]**. Howver, there is a catch. The Area ID values in the **Var1** column
of the data frame are not numeric, but **factors**. In R, a **factor** is used for categorical variables and its value in and of itself is meaningless (internally, the factors are turned into **levels** and become
consecutive values). Since there are only 75 different neighborhoods with counts, the factor **Var1** will only take 75 levels, and not the 77 that we need.

This situation illustrates how important it is in R to know the exact type of the variables you are working with. The 
best way to find out is to experiment, for example, by carrying out an **is.factor** test in **Var1** (remember to
use the dollar sign unless you have attached the data frame).

In [5]:
is.factor(pcframe$Var1)

[1] TRUE

In order to convert the factors to a numeric value that is not their level, we need to jump through some hoops. First, 
we turn the factor into its level, which is a character. Then we turn that character back into
a numeric value, so "24" becomes 24. These numeric values can then be used as row indices to insert the vehicle
counts in the correct position.

To keep things simple, we extract the factor as a new variable **nnf** and show the **levels**.

In [12]:
nnf <- pcframe$Var1
levels(nnf)

[1] "1"  "2"  "3"  "4"  "5"  "6"  "7"  "8"  "9"  "10" "11" "12" "13" "14" "15"
[16] "16" "17" "18" "19" "20" "21" "22" "23" "24" "25" "26" "27" "28" "29" "30"
[31] "31" "32" "33" "34" "35" "36" "37" "38" "39" "40" "41" "42" "43" "44" "45"
[46] "46" "48" "49" "50" "51" "52" "53" "55" "56" "57" "58" "59" "60" "61" "62"
[61] "63" "64" "65" "66" "67" "68" "69" "70" "71" "72" "73" "74" "75" "76" "77"

We now convert this to a numeric value using **as.numeric(levels( ))**. While this is not that efficient in
general, since each factor is unique in our case, that doesn't matter much.

In [13]:
nnn <- as.numeric(levels(nnf))
nnn

[1]  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
[26] 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 48 49 50 51
[51] 52 53 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77

In [14]:
is.factor(nnn)

[1] FALSE

We now initialize a vector of length 77 (the max value taken by the Community Area ID) to zero. Then we extract the
**Freq** column from the **pcframe** data frame and assign it to the row numbers that correspond to **nnn**.

In [15]:
narea <- max(vpoints$Commu_Area)
vc <- vector(mode="numeric",length=narea)
vc

[1] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[39] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[77] 0

In [16]:
length(vc)

[1] 77

In [17]:
vc[nnn] <- pcframe$Freq
vc

[1]  42  50  17  29  49  40  28   9  12  28  40   8  14  58 104  64  84  29  69
[20]  17  29  71  47  76  98  12  13  31  15  32  20  30   1   7  14   7   3  17
[39]  10  11   6   6  34  35  11  21   0  15  29   7  12  16  16   0   5  30  13
[58]  48  23  30  45  31  39  22  33  78  27  22  39  43  32   6  37   3  21   7
[77]  41

### Write csv file

We now have all the pieces to put together a data frame with the vehicle counts and write that out to a csv file.

We create a sequence from 1 to narea as the ID variable and then combine this with the vehicle counts **vc** into a 
data frame. To make sure the resulting dbf file will have the ID variable as an integer, we specify
**as.integer** to force the correct type. Also, we use **names** to give the columns better variable names. Finally, with **write.csv** we create the 
csv output file.

In [21]:
nid <- (1:narea)
vcframe <- data.frame(as.integer(nid),as.integer(vc))
vcframe

as.integer.nid.,as.integer.vc.
1,42
2,50
3,17
4,29
5,49
6,40
7,28
8,9
9,12
10,28


In [22]:
names(vcframe) <- c("AREAID","Vehicles")

In [23]:
write.csv(vcframe,"vehicle_counts.csv",row.names=FALSE)